# Boxplots for PDSW Paper

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 18})
plt.rcParams['image.cmap'] = 'gray'
import matplotlib.gridspec
import os

In [ ]:
import pandas
import numpy as np
import scipy
import scipy.stats as stats
import scipy.interpolate
import json
import datetime
import bisect
import warnings
import textwrap

### black magic necessary for processing Mira log files :(
try:
    import pytz
    _USE_TZ = True
except ImportError:
    _USE_TZ = False

def wrap(text, width=15):
    """wrapper for the wrapper"""
    return '\n'.join(textwrap.wrap(text=text,width=width))

In [ ]:
def utc_timestamp_to_argonne( timestamp ):
    """
    This is a batty function that allows us to compare the UTC-based
    timestamps from Darshan logs (start_time and end_time) to the
    Chicago-based YYYY-MM-DD dates used to index the mmdf data.
    """
    if _USE_TZ:
        ### we know that these logs are from Chicago
        tz = pytz.timezone("America/Chicago")
        
        ### Darshan log's start time in UTC, so turn it into a datetime with UTC on it
        darshan_time = pytz.utc.localize(datetime.datetime.utcfromtimestamp(timestamp))
        
        ### Then convert this UTC start time into a local start time so
        ### we can compare it to the local mmdf timestamp
        darshan_time_at_argonne = darshan_time.astimezone(tz)
        return darshan_time_at_argonne
    else:
        ### we assume that this script is running on Argonne time; it's the best we can do
        warnings.warn("pytz is not available so mmdf data might be misaligned by a day!")
        return datetime.datetime.fromtimestamp(timestamp)

In [ ]:
### Relative path to the repository's root directory
_REPO_BASE_DIR = os.path.join('..', '..')

### Translates cryptic counter names into something suitable for labeling plots
counter_labels = json.load(open(os.path.join(_REPO_BASE_DIR, 'scripts', 'counter_labels.json'), 'r'))

### For consistency, always plot file systems in the same order
_FILE_SYSTEM_ORDER = [ 'scratch1', 'scratch2', 'scratch3', 'mira-fs1' ]

_INPUT_EDISON_DATA_CSV = os.path.join(_REPO_BASE_DIR,
                                      'data',
                                      'dat',
                                      'tokio-lustre',
                                      'edison-abc-stats_2-14_3-28_v2.csv')
_INPUT_MIRA_DATA_CSV = os.path.join(_REPO_BASE_DIR,
                                    'data',
                                    'dat',
                                    'tokio-gpfs',
                                    'alcf-abc-stats_2-25_3-27.dat')
_INPUT_MIRA_MMDF_CSV = os.path.join(_REPO_BASE_DIR,
                                    'data',
                                    'dat',
                                    'tokio-gpfs',
                                    'mira_mmdf_1-25_3-27.csv')

_COVERAGE_FACTOR_CUTOFF = 1.2

_MIRA_JOBS_BLACKLIST = [ 1039807 ]

In [ ]:
### This is a time range that encompasses data collected on both Edison and Mira

# start time is inclusive
_START_TIME = datetime.datetime(2017, 2, 24, 0, 0, 0)
# end time is exclusive
_END_TIME   = datetime.datetime(2017, 3, 26, 0, 0, 0)
# _START_TIME = _END_TIME = None

In [ ]:
### All time

# start time is inclusive
_START_TIME = datetime.datetime(2016, 2, 24, 0, 0, 0)
# end time is exclusive
_END_TIME   = datetime.datetime(2017, 3, 26, 0, 0, 0)
# _START_TIME = _END_TIME = None

## Load data

We've been storing most of the per-job summary data in a single CSV per system.  We

1. Load the CSV directly into a dataframe
2. Drop any rows containing NANs, because if any of the core data is missing (e.g., application name), the whole record is useless.  Hopefully I haven't overlooked anything important in this assumption.
3. Synthesize a few new columns (we call these "metrics" in the paper) to facilitate downstream analysis

In [ ]:
### Edison
df_edison = pandas.DataFrame.from_csv(_INPUT_EDISON_DATA_CSV).dropna()
df_edison['darshan_rw'] = [ 'write' if x == 1 else 'read' for x in df_edison['darshan_write_mode?'] ]
df_edison['darshan_file_mode'] = [ 'shared' if x in ['H5Part','MPIIO'] else 'fpp' for x in df_edison['darshan_api'] ]
df_edison.rename(columns={'lmt_bytes_covered': 'coverage_factor'}, inplace=True)
df_edison['system'] = "edison"
df_edison['iops_coverage_factor'] = -1.0
df_edison['nodehr_coverage_factor'] = df_edison['job_num_nodes'] * \
                                      (df_edison['darshan_end_time'] - df_edison['darshan_start_time']) / 3600.0 / \
                                      (df_edison['job_concurrent_nodehrs'])


### Mira
df_mira = pandas.DataFrame.from_csv(_INPUT_MIRA_DATA_CSV).dropna()
rename_dict = { '# platform': "system" }
for key in df_mira.keys():
    if key == 'file_sys':
        rename_dict[key] = 'darshan_file_system'
    elif key not in rename_dict and not key.startswith('ggio_'):
        rename_dict[key] = 'darshan_' + key
df_mira.rename(columns=rename_dict, inplace=True)
df_mira['darshan_file_mode'] = [ 'shared' if x in ['H5Part','MPIIO'] else 'fpp' for x in df_mira['darshan_api'] ]
df_mira['coverage_factor'] = df_mira['darshan_total_bytes'] / (df_mira['ggio_bytes_read'] + df_mira['ggio_bytes_written'])
df_mira['iops_coverage_factor'] = (df_mira['darshan_total_rws'] / (df_mira['ggio_read_reqs'] + df_mira['ggio_write_reqs']))
df_mira['nodehr_coverage_factor'] = -1.0

Because I'm lazy, load the `mmdf` data separately and attach it to `df_mira`.  The mmdf CSV is generated by

1. retrieving all of the `df_fs1_*.txt` files from `mira:/projects/radix-io/automated/runs/gpfs-logs/`
2. running `tokio-cron-benchmarks:utils/parse_mmdf.py` script against `df_fs1_*.txt` (note that although `parse_mmdf.py` can distinguish between different file systems, this script currently doesn't filter for the correct `file_system` column

In [ ]:
df_mmdf = pandas.DataFrame.from_csv(_INPUT_MIRA_MMDF_CSV, index_col=['file_system', 'date'])
df_mmdf['free_kib'] = df_mmdf['free_kib_blocks'] + df_mmdf['free_kib_frags']
df_mmdf['free_pct'] = df_mmdf['free_kib'] / df_mmdf['disk_size']

Walk the master dataframe and attach mmdf data.  Note that we're injecting NAs for missing mmdf data because missing mmdf data should not exclude the entire day from our analysis.

In [ ]:
### I really hope iterrows behaves deterministically and preserves order...
new_data = {
    'mmdf_avg_fullness_pct': [],
    'mmdf_max_fullness_pct': [],
}

### iterate over each row of the master Mira dataframe
no_data = set([])
for row in df_mira.itertuples():
    fs_key = row.darshan_file_system
    mmdf_key = utc_timestamp_to_argonne( row.darshan_start_time ).strftime("%Y-%m-%d")
    if mmdf_key in df_mmdf.loc[fs_key].index:
        ### only look at today's data
        df = df_mmdf.loc[fs_key].loc[mmdf_key]
        
        data_cols = [ True if x else False for x in df['data?'] ]

        ### calculate a percent fullness - don't bother saving the id of this fullest server though
        new_data['mmdf_max_fullness_pct'].append( 1.0 - df[ data_cols ]['free_pct'].min() )
        new_data['mmdf_avg_fullness_pct'].append( 1.0 - df[ data_cols ]['free_pct'].mean() )
    else:
        no_data.add( datetime.datetime.fromtimestamp(row.darshan_start_time).strftime("%Y-%m-%d") )
        new_data['mmdf_max_fullness_pct'].append( np.nan )
        new_data['mmdf_avg_fullness_pct'].append( np.nan )

warnings.warn("No MMDF data found for the following dates:\n" + '\n'.join(no_data))
        
for new_col_name, new_col_data in new_data.iteritems():
    df_mira[new_col_name] = new_col_data

Now merge both DataFrames so we can look at all the data if we really want to.  This DataFrame will have a bunch of NANs for data that is only applicable to Mira or Edison.

In [ ]:
df_concat = pandas.concat( (df_mira, df_edison) )

## Filter Data

Two notable filters are applied:

1. All jobs where the bytes coverage factor and ops coverage factor are greater than 1.2 are discarded because they reflect severely misaligned or gappy data.

2. Mira job 1039807 is excluded because ggiostat returned highly abnormal results starting that day.  See e-mail from Shane and Phil on March 23 about this.

3. All Edison jobs from March 12 were discarded because LMT broke as a result of daylight saving time rolling over.  This filter was applied _before_ the input CSV files loaded above were generated, so it does not need to be applied here.

In [ ]:
for df in df_mira, df_edison, df_concat:
    print '+'.join( df['system'].unique() )
    print ''.join([ '=' * 50])
    
    count = len(df)
    df.drop(df.index[df['coverage_factor'] > _COVERAGE_FACTOR_CUTOFF], inplace=True)
    print "Dropped %d records due to coverage factor > %.1f" % (count - len(df), _COVERAGE_FACTOR_CUTOFF)
    
    count = len(df)
    for blacklisted_job in _MIRA_JOBS_BLACKLIST:
        df.drop(df.index[(df['system'] == 'mira') & (df['darshan_jobid'] == blacklisted_job)], inplace=True)
    print "Dropped %d records due to Mira job #1039807" % (count - len(df))

    count = len(df)
    if _END_TIME is not None and _START_TIME is not None:
        filter_time = [ datetime.datetime.fromtimestamp(df['darshan_start_time'][x]) < _START_TIME                    
                        or
                        datetime.datetime.fromtimestamp(df['darshan_start_time'][x]) > _END_TIME
                        for x in df.index ]
        df.drop(df.index[filter_time], inplace=True)
    print "Dropped %d records outside of time range %s - %s" % (count - len(df), _START_TIME, _END_TIME)
    print "Total measurements to be analyzed:", len(df)
    print

In [ ]:
for df in df_mira, df_edison, df_concat:
    print '+'.join( df['system'].unique() )
    print ''.join([ '=' * 50])
    print "Earliest timestamp now", datetime.datetime.fromtimestamp(df['darshan_start_time'].min())
    print "Latest timestamp now", datetime.datetime.fromtimestamp(df['darshan_start_time'].max())
    print

## Normalize Performance
Different file systems, benchmarks, and read/write modes are capable of different peak bandwidths.  As such, we want to normalize the absolute performance (`summarize_key`) by something.  For convenience we calculate the denominator for normalization a couple of different ways (e.g., the mean, median, and max measurement).  We also limit normalization to unique combinations of variables specified by `normalization_group` below.

Calculate the normalization factors (the denominators), then apply that factor to all of the data in the DataFrame.  These normalized data will be saved as new columns in the DataFrame.

In [ ]:
### The actual variable we want to normalize
summarize_key = 'darshan_agg_perf_by_slowest'

for df in df_edison, df_mira, df_concat:
    ### Specify which keys we want to group together before normalizing
    normalization_group = df.groupby(['darshan_app', 'darshan_file_system', 'darshan_file_mode', 'darshan_rw'])

    ### Dict to store the denominators for normalization
    normalization_data = {
        'mean':   normalization_group.darshan_agg_perf_by_slowest.mean(),
        'median': normalization_group.darshan_agg_perf_by_slowest.median(),
        'max':    normalization_group.darshan_agg_perf_by_slowest.max(),
    }

    ### Normalize every row in the DataFrame by all of our denominators
    new_cols = {}
    for func in normalization_data.keys():
        new_col_key = 'darshan_normalized_perf_by_%s' % func
        new_cols[new_col_key] = []
        for index, row in df.iterrows():
            new_cols[new_col_key].append(
                row[summarize_key] / normalization_data[func]
                                                       [row['darshan_app']]
                                                       [row['darshan_file_system']]
                                                       [row['darshan_file_mode']]
                                                       [row['darshan_rw']]
            )

    ### Also just do per-file system
    normalization_group = df.groupby('darshan_file_system')
    normalization_data = {
        'mean':   normalization_group.darshan_agg_perf_by_slowest.mean(),
        'median': normalization_group.darshan_agg_perf_by_slowest.median(),
        'max':    normalization_group.darshan_agg_perf_by_slowest.max(),
    }
    for func in normalization_data.keys():
        new_col_key = 'darshan_normalized_perf_by_fs_%s' % func
        new_cols[new_col_key] = []
        for index, row in df.iterrows():
            new_cols[new_col_key].append(
                row[summarize_key] / normalization_data[func][row['darshan_file_system']])

    ### Take our normalized data and add them as new columns
    for new_col, new_col_data in new_cols.iteritems():
        df[new_col] = new_col_data

## Multivariate Correlation Analysis
`performance_key` is the variable we wish to use to represent performance.  It is typically

- `darshan_agg_perf_by_slowest`, which is the absolute performance measured by each benchmark run
- `darshan_normalized_perf_by_max`, which is normalized by the maximum observed performance
- `darshan_normalized_perf_by_mean`, which is normalized by the mean observed performance

In [ ]:
performance_key = 'darshan_normalized_perf_by_max'

### Numerical Correlation Analysis

Pearson analysis assumes that each variable is normally distributed.  It is easier to understand, but it is not technically correct for variables that are _not_ normally distributed, which include performance.  The Spearman coefficient would be better and can be enabled below.

We use `scipy.stats` to calculate p-values associated with each correlation.  The ultimate artifact of this process is a table of interesting correlations, their correlation coefficients, and color coding to indicate the confidence of those coefficients based on p-values.

In [ ]:
ignore_cols = [
    'lmt_tot_zeros',
    'lmt_frac_zeros',
    'lmt_frac_missing',
    'ost_avg_kib',
    'ost_min_pct',
    'ost_min_kib',
    'ost_max_kib',
    'ost_count',
#   'ost_bad_ost_count',
    'ost_bad_ost_pct',
    'ost_failures_lead_secs',
    'ost_fullness_lead_secs',
    'lmt_tot_missing',
    'ost_avg_bad_ost_per_oss',
    'ost_avg_bad_overload_factor',
    'ost_bad_oss_count',
    'ost_min_id',
    'ost_max_id',
    'job_min_radius',
    'job_avg_radius',
### second pass
    'lmt_ops_getattrs',
    'lmt_ops_getxattrs',
    'lmt_ops_rmdirs',
    'lmt_ops_unlinks',
    'lmt_ops_renames',
    'lmt_ops_setattrs',
    'lmt_ops_mkdirs',
    'ggio_inoded_updates',
### third pass
    "lmt_mds_ave",
    "lmt_oss_ave",
]

### if one key has the same logical meaning as another, this will remap those
### keys so they line up in the DataFrame
equivalent_keys = {
    'ggio_closes':     'lmt_ops_closes',
    'ggio_opens': 'lmt_ops_opens',
    'ggio_bytes_read': 'lmt_tot_bytes_read',
    'ggio_bytes_written': 'lmt_tot_bytes_write',
    'mmdf_max_fullness_pct': 'ost_max_pct',
    'mmdf_avg_fullness_pct': 'ost_avg_pct',
}

### Specific names for the table
counter_labels_table = {
    'coverage_factor': "Coverage Factor (Bandwidth)",
    'nodehr_coverage_factor': "Coverage Factor (NodeHrs)",
    "ost_avg_pct": "Avg LUN Fullness",
    "ost_max_pct": "Fullness on Fullest LUN",
    "lmt_oss_max": "Max CPU Load, Data Server",
    "ost_bad_pct": "% Servers Failed Over",
    "ost_bad_ost_count": "Failed-over Servers",
    "lmt_ops_closes": "close(2) Calls",
    "lmt_ops_opens": "open(2) Calls",
    "lmt_tot_bytes_write": "Bytes Written",
    "lmt_tot_bytes_read": "Bytes Read",
    "lmt_mds_max": "Max CPU Load, Metadata Server",
    "lmt_mds_ave": "Avg CPU Load, Metadata Server",
    "job_concurrent_jobs": "# Concurrent Jobs",
    "lmt_oss_ave": "Avg CPU Load, Data Server",
    "job_max_radius": "Job Diameter",
    "iops_coverage_factor": "Coverage Factor (IOPs)",
    "ggio_write_reqs": "Write Ops",
    "ggio_read_reqs": "Read Ops",
    "ggio_read_dirs": "readdir(3) Calls",
}

### Order in which table is to be printed
print_order = [
    'coverage_factor',
    'nodehr_coverage_factor',
    "iops_coverage_factor",
    "lmt_ops_closes",
    "lmt_ops_opens",
#   "ggio_read_dirs",
    "lmt_tot_bytes_write",
    "lmt_tot_bytes_read",
    "ggio_write_reqs",
    "ggio_read_reqs",
    "ost_max_pct",
    "ost_avg_pct",
    "lmt_mds_max",
    "lmt_oss_max",
    "ost_bad_ost_count",
    "job_concurrent_jobs",
#   "job_max_radius",
]

## Distribution of each benchmark type

Following cell defines which variable we wish to aggregate into boxplots and a few plotting parameters that depend on our choice of variable.

In [ ]:
boxplot_settings = {
    'fontsize': 15,
    'darshan_normalized_perf_by_fs_max': {
        'output_file': "perf-boxplots-per-fs.pdf",
        'ylabel': "Fraction of Peak\nFile System Performance",
        'title_pos': [ 
            {'x': 0.97, 'y': 0.80, 'horizontalalignment': 'right', 'fontsize': 14},
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14}]
    },
    'darshan_normalized_perf_by_max': {
        'output_file': "perf-boxplots.pdf",
        'ylabel': "Fraction of Peak Performance",
        'title_pos': [ 
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14},
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14}]
    },
}

We plot two sets of boxplots based on the normalization denominator:

1. normalized by the maximum without any grouping (other than file system)
2. normalized by the maximum of each unique combination of app-read/write-filemode

The idea is to show that

1. performance variation varies across different file systems and different applications
2. even within an application, the magnitude of performance variation varies with file system

In [ ]:
def create_boxplox(df_concat, fs, plot_variable, ax, irow, icol, other_settings={}):
    if ax is None:
        fig, ax = plt.subplots()
        separate = True
        other_settings.update({
            'boxprops': {'linewidth': 2},
            'medianprops': {'linewidth': 4},
            'whiskerprops': {'linewidth': 2},
            'capprops': {'linewidth': 2},
        })
    else:
        separate = False
        other_settings.update({'medianprops': {'linewidth':2}})
    boxplot_group_by = [ 'darshan_file_mode', 'darshan_rw', 'darshan_app' ]
    df_concat.loc[df_concat["darshan_file_system"] == fs]\
    .boxplot(
        column=[plot_variable],
        by=boxplot_group_by,
        ax=ax,
        widths=0.75,
        whis=[5,95],
        showfliers=False,
        **(other_settings))

    if separate:
        settings = {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 24}
    else:
        settings = boxplot_settings[plot_variable]['title_pos'][irow]
    title = ax.set_title(fs, **(settings))
    title.set_bbox({'color': 'white', 'alpha': 0.5})
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.xaxis.grid(True)

    ### relabel the x axis labels
    new_labels = []
    for axis_label in ax.get_xticklabels():
        current_label = axis_label.get_text()
        axis_label.set_rotation(90)
        if "IOR" in current_label:
            if "shared" in current_label:
                new_label = "IOR/shared"
            else:
                new_label = "IOR/fpp"
        elif 'BD-CATS' in current_label:
            new_label = "BD-CATS"
        else:
            new_label = current_label.split(',')[2].strip(')').strip().split('-')[0]
        if 'write' in current_label:
            new_label += "(W)"
        else:
            new_label += "(R)"
        new_labels.append(new_label)

    ### set x tick labels for only the bottom row
    if irow == 0 and not separate:
        ax.set_xticklabels([])
    else:
        ax.set_xticklabels(new_labels,
                           fontsize=boxplot_settings['fontsize'])
    if separate:
        ax.yaxis.set_ticks( np.linspace(0.0, 1.0, 6) )
    else:
        ax.yaxis.set_ticks( np.linspace(0.0, 1.0, 3) )
    ax.set_ylim([-0.1, 1.1])
    for ytick in ax.yaxis.get_major_ticks():
        ytick.label.set_fontsize(boxplot_settings['fontsize'])
        
    if separate:
        return fig, ax

In [ ]:
for plot_variable in performance_key, performance_key.replace('_by_', '_by_fs_'):
    fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True)
    fig.set_size_inches(8,6)
    for idx, fs in enumerate(_FILE_SYSTEM_ORDER):
        icol = idx / 2
        irow = idx % 2
        ax = axes[irow, icol]
        create_boxplox(df_concat, fs, plot_variable, ax, irow, icol)
        
    fig.suptitle("")
    # fig.text(0.5, -0.4, 'common X', ha='center')
    fig.text(0.0, 0.5,
             boxplot_settings[plot_variable]['ylabel'],
             verticalalignment='center',
             horizontalalignment='center',
             rotation='vertical',
             fontsize=boxplot_settings['fontsize'])
    fig.subplots_adjust(hspace=0.05, wspace=0.05)

    output_file = boxplot_settings[plot_variable]['output_file']
    fig.savefig(output_file, bbox_inches="tight")
    print "Saved %s" % output_file

Break out each box plot as its own standalone plot for the purposes of the powerpoint presentation

In [ ]:
boxplot_settings = {
    'fontsize': 20,
    'darshan_normalized_perf_by_max': {
        'output_file': "perf-boxplots.pdf",
        'ylabel': "Fraction of\nPeak Performance",
        'title_pos': [ 
            {'x': 0.97, 'y': 0.80, 'horizontalalignment': 'right', 'fontsize': 14},
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14}]
    },
}

In [ ]:
for plot_variable in [performance_key]:
    for idx, fs in enumerate(_FILE_SYSTEM_ORDER):
        fig, ax = create_boxplox(df_concat, fs, plot_variable, None, 0, 0)
        ax.set_xticklabels(ax.get_xticklabels(), fontsize=boxplot_settings['fontsize']*0.9, rotation=30, ha='right')
        fig.suptitle("")
        fig.text(-0.05, 0.5,
                 boxplot_settings[plot_variable]['ylabel'],
                 verticalalignment='center',
                 horizontalalignment='center',
                 rotation='vertical',
                 fontsize=boxplot_settings['fontsize']*0.9)
        fig.subplots_adjust(hspace=0.05, wspace=0.05)

        output_file = boxplot_settings[plot_variable]['output_file'].replace('.pdf', '_%s.pdf' % fs)
        fig.savefig(output_file, bbox_inches="tight")
        print "Saved %s" % output_file

In [ ]:
plot_variable = 'darshan_normalized_perf_by_max'

boxplot_settings = {
    'fontsize': 20,
    'darshan_normalized_perf_by_max': {
        'output_file': "perf-boxplots-x3.pdf",
        'ylabel': "Fraction of\nPeak Performance",
        'title_pos': [ 
            {'x': 0.97, 'y': 0.80, 'horizontalalignment': 'right', 'fontsize': 14},
            {'x': 0.04, 'y': 0.02, 'horizontalalignment': 'left', 'fontsize': 14}]
    },
}

fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True)
fig.set_size_inches(14,4)
for icol, fs in enumerate(['mira-fs1', 'scratch3', 'scratch2']):
    ax = axes[icol]
    create_boxplox(df_concat, fs, plot_variable, ax, irow, icol)
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=boxplot_settings['fontsize']*0.8, rotation=30, ha='right')
    fig.suptitle("")
#   fig.text(-0.05, 0.5,
#            boxplot_settings[plot_variable]['ylabel'],
#            verticalalignment='center',
#            horizontalalignment='center',
#            rotation='vertical',
#            fontsize=boxplot_settings['fontsize']*0.9)
    fig.subplots_adjust(hspace=0.05, wspace=0.05)

fig.suptitle("")
# fig.text(0.5, -0.4, 'common X', ha='center')
fig.text(0.03, 0.5,
         boxplot_settings[plot_variable]['ylabel'],
         verticalalignment='center',
         horizontalalignment='center',
         rotation='vertical',
         fontsize=boxplot_settings['fontsize']*0.9)
fig.subplots_adjust(hspace=0.05, wspace=0.05)

output_file = boxplot_settings[plot_variable]['output_file']
fig.savefig(output_file, bbox_inches="tight", transparent=True)
print "Saved %s" % output_file